In [1]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

import re
import pandas as pd
import xlrd

In [3]:
result_4 = pd.read_excel('2020.04.01-2020.04.31_result.xlsx')

In [4]:
result_4.head()

,Unnamed: 0,years,company,title,contents,link
0,0,2020.04.28.,연합뉴스,"길어진 사회적 거리두기에…성인 80% ""피로감 느낀다""","직장인 황금연휴 계획 1, 2위는 실내 휴식·계획 없음한강에서도 사회적 거리 유지 ...",https://news.naver.com/main/read.nhn?mode=LSD&...
1,1,2020.04.28.,donga.com,"“사회적 거리두기, ‘이것’ 때문에 피로감 느낀다”",사회적 거리두기 기간이 연장되면서 피로감을 느끼는 사람이 10명중 8명에 달한다는 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2,2020.04.28.,financial news,"성인남녀 10명 중 8명, 사회적 거리두기 피로감 느껴",독서모임 회원들이 모여 활동을 하고 있다. 뉴스1 제공 [파이낸셜뉴스] 코로나19 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,3,2020.04.26.,media KHAN,황금연휴 여행 수요 ‘들썩’…거리 두기 이완 우려,ㆍ5월 연휴 앞두고 항공·철도표 매진ㆍ“예약률만 보면 코로나 이전 수준”주말 나들이...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,4,2020.04.19.,financial news,5월5일까지 사회적 거리두기 유지,"정 총리 ""연휴 유동인구 급증 예고""야외 공공시설·무관중 경기는 허용 중대본 영상회...",https://news.naver.com/main/read.nhn?mode=LSD&...


In [5]:
del result_4['Unnamed: 0']

In [6]:
result_4 = result_4[result_4['link'].notnull()]

In [7]:
url="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=018&aid=0004300484&date=20190128&type=1&rankingSeq=5&rankingSectionId=101"    

header = {
    "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36", 
    "referer": url,  
}

In [13]:
# link url 리스트
url = []
for l in result_4['link']:    
    url.append(l)

# link url 파싱
oid = []
aid = []
for i in url:
    oid.append(i.split("oid=")[1].split("&")[0]) 

for j in url:
    aid.append(j.split("aid=")[1])
        
# result_path
RESULT_PATH = '/home/jeehyun.lee.conda/'

c_List=[] 
d_List=[]

In [ ]:
def contents_flat(c_List,d_List): 
    flatList = [] 
    d_flatList=[]
    #f = open("C:/Users/User/Desktop/python study/beautifulSoup_ws/crawling_result/comment_text.txt", 'w', encoding='utf-8')
    for elem in c_List: 
        # if an element of a list is a list 
        # iterate over this list and add elements to flatList  
        
        #elem=re.sub('"',' ',str(elem))
        if type(elem) == list: 
            for e in elem: 
                flatList.append(e) 
        else: 
            flatList.append(elem) 
           
    for elem in d_List: 
        if type(elem) == list: 
            for e in elem: 
                e=re.sub('\\+0900',' ',e)
                d_flatList.append(e) 
        else: 
            elem=re.sub('\\+0900',' ',elem)
            d_flatList.append(elem) 

    #f.write(str(flatList))
    excel_make(flatList,d_flatList)
             
def excel_make(flatList,d_flatList): 
    result= {"comments":flatList, "date":d_flatList}
    df = pd.DataFrame.from_dict(result, orient = 'index')  # df로 변환
    df = df.T
    df.to_excel(RESULT_PATH+"comments_result_4.xlsx", sheet_name = 'sheet1')

def main():
    for o,a in [(oid, aid)]:
        for i in range(len(url)):
            page = 1
            comment_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+o[i]+"%2C"+a[i]+"&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE"
            print(comment_url)
            r=requests.get(comment_url,headers=header)
            cont=bs(r.content,"html.parser")     
            total_comm=str(cont).split('comment":')[1].split(",")[0]
            print(total_comm)
            
            match=re.findall('"contents":"([^\*]*)","userIdNo"', str(cont))   #소괄호 안에 있는것을 크롤링해온다.
            date=re.findall('"modTime":"([^\*]*)","modTimeGmt"', str(cont))

            c_List.append(match)
            d_List.append(date)
            
            while True:
                if int(total_comm) <= ((page) * 20): 
                    break
                else:
                    page += 1
                    
        print(len(c_List))
        print(len(d_List))
 
    contents_flat(c_List,d_List)  
    
main()

https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news001%2C0011575612&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&refresh=false&sort=FAVORITE
209
https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news020%2C0003283356&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&refresh=false&sort=FAVORITE
17
https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news014%2C0004415668&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&refresh=false&sort=FAVORITE
2
https://apis.naver.com/comme